In [ ]:
import requests
import pandas as pd
from collections import defaultdict

def get_population_dep(start_year=2016, end_year=None):
    if end_year is None:
        import datetime
        end_year = datetime.datetime.now().year

    # dictionnaire temporaire pour agréger
    agg = defaultdict(int)

    for year in range(start_year, end_year + 1):
        params = {
            "GEO": "DEP",
            "TIME_PERIOD": str(year),
            "maxResult": "500"
        }

        response = requests.get("https://api.insee.fr/melodi/data/DS_ESTIMATION_POPULATION", params=params)
        if response.status_code != 200:
            print(f"Erreur API pour l'année {year}: {response.status_code}")
            continue

        data = response.json()
        obs = data.get("observations", [])

        for o in obs:
            geo = o["dimensions"]["GEO"]

            # ne garder que les départements
            if "DEP-" not in geo:
                continue
            code_dep = geo.split("DEP-")[1]
            year_obs = int(o["dimensions"]["TIME_PERIOD"])

            # récupérer la valeur
            val = o["measures"]["OBS_VALUE_NIVEAU"].get("value")
            if val is not None:
                # agrégation par département et année
                agg[(code_dep, year_obs)] += int(val)

    # transformer en DataFrame
    records = [{"code": k[0], "year": k[1], "population": v} for k, v in agg.items()]
    df_pop = pd.DataFrame(records)
    df_pop = df_pop.sort_values(by=["year", "code"]).reset_index(drop=True)
    return df_pop
